In [1]:
#Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Install
!pip install catboost
!pip install xlsxwriter
!pip install catboost
!pip install scikit-optimize
!pip install xgboost
!pip install lightgbm
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 6.7 MB/s eta 0:00:00


In [3]:
import os
import pandas as pd
from numpy import mean
from numpy import std
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import *
from threading import Thread
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.base import BaseEstimator, ClassifierMixin
from csv import writer

from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.exceptions import ConvergenceWarning
import csv
import time
import warnings


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [7]:
def load_spectrum_file(dir_path, max_subfolders=None, max_files_per_subdir=None, max_rows=None):
    """Carrega arquivos de espectro e associa rótulos numéricos às subpastas."""
    data, labels = [], []
    subfolder_labels = {}
    label_encoder = LabelEncoder()

    # Contadores para controlar a quantidade de subpastas e arquivos lidos
    subfolder_count = 0

    for root, dirs, files in os.walk(dir_path):
        # Limitar a quantidade de subpastas
        if max_subfolders is not None and subfolder_count >= max_subfolders:
            break

        for dir_name in dirs:
            subfolder_path = os.path.join(root, dir_name)
            subfolder_labels[subfolder_path] = dir_name
            subfolder_count += 1

            # Limitar a quantidade de subpastas
            if max_subfolders is not None and subfolder_count > max_subfolders:
                break

            file_count = 0
            for filename in os.listdir(subfolder_path):
                if filename.endswith('.csv'):
                    if max_files_per_subdir is not None and file_count >= max_files_per_subdir:
                        break
                    file_path = os.path.join(subfolder_path, filename)
                    try:
                        df = pd.read_csv(file_path, delimiter=',', nrows=max_rows)
                        y = df.iloc[:, 1].values  # Pega os valores da segunda coluna

                        # Adiciona o rótulo da subpasta à lista de labels
                        labels.append(subfolder_labels[subfolder_path])
                        data.append(y)
                        file_count += 1
                    except Exception as e:
                        print(f"Error reading file {file_path}: {e}")

    # Transforma os rótulos das subpastas em números
    labels = label_encoder.fit_transform(labels)
    return np.array(data), np.array(labels), label_encoder.classes_

# Exemplo de uso
print("Loading data...")
data, labels, class_names = load_spectrum_file(
    '/content/drive/MyDrive/Iniciação Científica 2023-2024/PQ_TCC_IC/Raman/Data_csv/',
    max_subfolders=None,
    max_files_per_subdir=None,
    max_rows=None
)
print("                                   ")
print("^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^")
print("                                   ")
print(f"Data shape: {data.shape}")
print(f"Labels shape: {labels.shape}")
print(f"Class names: {class_names}")
print("                                   ")
print("^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^")
print("                                   ")

X_=data
y=labels

#train_data, test_data, train_label, test_label = train_test_split(data, labels, test_size=30, train_size=70, random_state=1, shuffle=True, stratify=True)
#print(f"Train data shape: {train_data.shape}, Train label shape: {train_label.shape}")
#print(f"Test data shape: {test_data.shape}, Test label shape: {test_label.shape}")
#print("                                   ")
#print("^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^")
#rint("                                   ")
#X_train = train_data
#y_train = train_label
#X_test = test_data
#y_test = test_label

#print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
#print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")
#print("                                   ")
#print("^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^")
#print("                                   ")
#print("Completed!")

Loading data...
                                   
^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^ ^¬^
                                   
Data shape: (5292, 1570)
Labels shape: (5292,)
Class names: ['Actinolite' 'Adamite' 'Aegirine' 'Aenigmatite' 'Akermanite' 'Albite'
 'Almandine' 'Amblygonite' 'Amesite' 'Analcime' 'Anatase' 'Andalusite'
 'Andradite' 'Anglesite' 'Anhydrite' 'Ankerite' 'Annite' 'Anorthite'
 'Aragonite' 'Arfvedsonite' 'Arsenopyrite' 'Astrophyllite' 'Atacamite'
 'Augelite' 'Augite' 'AxiniteFe' 'Azurite' 'Baddeleyite' 'Barrerite'
 'Barysilite' 'Baryte' 'Bertrandite' 'Beryl' 'Beudantite' 'Bikitaite'
 'Bobdownsite' 'Braunite' 'Brazilianite' 'Brookite' 'Brucite' 'Bustamite'
 'Calcite' 'Cassiterite' 'Catapleiite' 'Celestine' 'Cerussite'
 'ChabaziteCa' 'Chloritoid' 'Chondrodite' 'Chrysoberyl' 'Clinochlore'
 'Clinohumite' 'ClinoptiloliteCa' 'Clinozoisite' 'ColumbiteFe'
 'Conichalcite' '

In [ ]:
print(labels)
print(np.unique(labels))
print(class_names)

[ 4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  3  3  3  3  3  3  3  3
  3  3  3  3  3  3  3  3  3  3  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  8  8  8  8  8  8  8  8  8  8  7  7  7  7  7  7  7  7  7  7
  7  7  7  7  7  7  7  7  7  7  7  7  7  7  5  5  5  5  5  5  5  5  5  5
  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5
  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5
  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5
  5  5  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  6  6  6  6  6  6  6  6  6  6  6  6

In [ ]:
def experiment(model_name, model, params, X_train, y_train, X_test, y_test, i):
    data_path = '/content/drive/MyDrive/Iniciação Científica 2023-2024/PQ_TCC_IC/Raman/Results/Clássicos/'
    # Configure the cross-validation procedure
    cv_inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)

    # Define search
    search = BayesSearchCV(model, params, scoring='accuracy', cv=cv_inner, n_iter=5, refit=True, random_state=1, n_jobs=1)

    # Execute search
    result = search.fit(X_train, y_train)

    # Get the best performing model fit on the whole training set
    best_model = result.best_estimator_

    # Evaluate model on the hold out dataset
    yhat = best_model.predict(X_test)

    # Evaluate the model
    acc = accuracy_score(y_test, yhat)
    prec = precision_score(y_test, yhat, average='weighted')
    rec = recall_score(y_test, yhat, average='weighted')
    f1 = f1_score(y_test, yhat, average='weighted')
    mcc = matthews_corrcoef(y_test, yhat)

    # Store the result
    results = [model_name, i, acc, rec, prec, f1, mcc, result.best_score_, result.best_params_]

    # Write the results to a CSV file in the specified directory
    results_file = f"{data_path}{model_name}_results.csv"
    with open(results_file, 'a', newline='') as f_object:
        writer_object = csv.writer(f_object)
        writer_object.writerow(results)

    # Report progress
    print(f"{model_name} {i} > acc={acc:.2f}, est={result.best_score_:.2f}, cfg={result.best_params_}")


In [17]:
# Definition of the experiment
def experiment(model_name, model, params, X_, y):
    # Caminho da pasta para salvar o arquivo
    data_path = '/content/drive/MyDrive/Iniciação Científica 2023-2024/PQ_TCC_IC/Raman/Results/Clássicos/SIEPE/'

    # Nome do arquivo para salvar os resultados
    results_file = os.path.join(data_path, model_name + '.xlsx')

    results = []

    # Configure o train_test_split fora do loop
    X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.3, random_state=1, stratify=y)

    # configure the cross-validation procedure para o treinamento
    cv_inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)

    # define search
    search = BayesSearchCV(model, params, scoring='accuracy', cv=cv_inner, n_iter=10, refit=True, random_state=1, n_jobs=-1)

    # execute search
    result = search.fit(X_train, y_train)

    # get the best performing model fit on the whole training set
    best_model = result.best_estimator_

    # evaluate model on the hold out dataset
    yhat = best_model.predict(X_test)

    # evaluate the model
    acc = accuracy_score(y_test, yhat)
    prec = precision_score(y_test, yhat, average='weighted', zero_division=1)
    rec = recall_score(y_test, yhat, average='weighted')
    f1 = f1_score(y_test, yhat, average='weighted')
    mcc = matthews_corrcoef(y_test, yhat)

    # store the result
    results.append([model_name, acc, rec, prec, f1, result.best_score_, result.best_params_])

    # report progress
    print(f"{model_name} > acc={acc:.3f}, est={result.best_score_:.3f}, cfg={result.best_params_}")

    # summarize the estimated performance of the model
    mean_acc = sum(r[1] for r in results) / len(results)
    mean_rec = sum(r[2] for r in results) / len(results)
    mean_prec = sum(r[3] for r in results) / len(results)
    mean_f1 = sum(r[4] for r in results) / len(results)

    # save results to file
    df = pd.DataFrame(results, columns=['model', 'acc', 'rec', 'prec', 'f1', 'best_score', 'best_params'])
    df.to_excel(results_file, index=False)

In [18]:
# Definição dos modelos e parâmetros
model_params = {
    'lr': {'model': LogisticRegression(),
           'params': {
               'C': Real(1e-4, 1e4, prior='log-uniform'),
               'fit_intercept': Categorical([True, False]),
               'solver': Categorical(['newton-cg', 'liblinear', 'saga'])}},  # Removido 'sag'

#    'knn': {'model': KNeighborsClassifier(),
#            'params': {
#                'n_neighbors': Integer(1, 50),
#                'weights': Categorical(['uniform', 'distance']),
#                'algorithm': Categorical(['auto', 'ball_tree', 'kd_tree', 'brute']),
#                'p': Integer(1, 5)}},
#
    'nb': {'model': GaussianNB(),
           'params': {
               'var_smoothing': Real(1e-10, 1e-1, prior='log-uniform')}},

    'dt': {'model': DecisionTreeClassifier(),
           'params': {
               'criterion': Categorical(['gini', 'entropy']),
               'splitter': Categorical(['best', 'random']),
               'max_depth': Integer(3, 30),
               'min_samples_split': Integer(2, 10),
               'min_samples_leaf': Integer(1, 10),
               'max_features': Real(0.1, 1.0, prior='uniform')}},

#    'svm': {'model': LinearSVC(),
#            'params': {
#                'C': Real(1e-6, 1e+6, prior='log-uniform'),
#                'loss': Categorical(['hinge', 'squared_hinge']),
#                'tol': Real(1e-6, 1e-2, prior='log-uniform')}},
#
#    'gpc': {'model': GaussianProcessClassifier(),
#            'params': {
#                'optimizer': Categorical(['fmin_l_bfgs_b', None]),
#                'n_restarts_optimizer': Integer(0, 10),
#                'max_iter_predict': Integer(100, 1000)}},

#    'mlp': {'model': MLPClassifier(),
#            'params': {
#                'hidden_layer_sizes': Integer(10, 100),
#                'activation': Categorical(['identity', 'logistic', 'tanh', 'relu']),
#                'solver': Categorical(['sgd', 'adam']),
#                'alpha': Real(1e-5, 1e-1, prior='log-uniform'),
#                'learning_rate': Categorical(['constant', 'invscaling', 'adaptive']),
#                'learning_rate_init': Real(1e-4, 1e-1, prior='log-uniform'),
#                'max_iter': Integer(1000, 1001)}},

#    'ridge': {'model': RidgeClassifier(),
#              'params': {
#                  'alpha': Real(1e-4, 1e4, prior='log-uniform'),
#                  'fit_intercept': Categorical([True, False]),
#                  'solver': Categorical(['auto', 'cholesky', 'lsqr', 'sparse_cg', 'saga'])}},  # Removido 'sag'

    'rf': {'model': RandomForestClassifier(),
           'params': {
               'n_estimators': Integer(10, 500),
               'criterion': Categorical(['gini', 'entropy']),
               'max_depth': Integer(3, 30),
               'min_samples_split': Integer(2, 10),
               'min_samples_leaf': Integer(1, 10),
               'max_features': Real(0.1, 1.0, prior='uniform'),
               'bootstrap': Categorical([True, False]),
               'class_weight': Categorical(['balanced', 'balanced_subsample', None])}},

#    'qda': {'model': QuadraticDiscriminantAnalysis(),
#            'params': {
#                'reg_param': Real(0, 1, prior='uniform'),
#                'store_covariance': Categorical([True, False]),
#                'tol': Real(1e-5, 1e-1, prior='log-uniform')}},

#    'ada': {'model': AdaBoostClassifier(),
#            'params': {
#                'n_estimators': Integer(10, 500),
#                'learning_rate': Real(1e-3, 1, prior='log-uniform'),
#                'algorithm': Categorical(['SAMME', 'SAMME.R'])}},

#    'gbc': {'model': GradientBoostingClassifier(),
#            'params': {
#                'n_estimators': Integer(10, 500),
#                'learning_rate': Real(1e-3, 1, prior='log-uniform'),
#                'max_depth': Integer(3, 10),
#                'min_samples_split': Integer(2, 10),
#                'min_samples_leaf': Integer(1, 10),
#                'max_features': Real(0.1, 1.0, prior='uniform'),
#                'subsample': Real(0.1, 1.0, prior='uniform')}},

#    'lda': {'model': LinearDiscriminantAnalysis(),
#            'params': {
#                'solver': Categorical(['lsqr', 'eigen']),
#                'shrinkage': Real(0, 1, prior='uniform'),
#                'tol': Real(1e-6, 1e-4, prior='log-uniform')}},
#
#    'et': {'model': ExtraTreesClassifier(),
#           'params': {
#               'n_estimators': Integer(10, 500),
#               'criterion': Categorical(['gini', 'entropy']),
#               'max_depth': Integer(3, 30),
#               'min_samples_split': Integer(2, 10),
#               'min_samples_leaf': Integer(1, 10),
#               'max_features': Real(0.1, 1.0, prior='uniform'),
#               'bootstrap': Categorical([True, False]),
#               'class_weight': Categorical(['balanced', 'balanced_subsample', None])}},

#    'xgboost': {'model': XGBClassifier(),
#                'params': {
#                    'learning_rate': Real(0.01, 0.3, prior='uniform'),
#                    'n_estimators': Integer(50, 500),
#                    'max_depth': Integer(3, 10),
#                    'min_child_weight': Integer(1, 10),
#                    'gamma': Real(0, 1, prior='uniform'),
#                    'subsample': Real(0.5, 1, prior='uniform'),
#                    'colsample_bytree': Real(0.5, 1, prior='uniform'),
#                    'reg_alpha': Real(0, 1, prior='uniform'),
#                    'reg_lambda': Real(1, 3, prior='uniform'),
#                    'scale_pos_weight': Real(1, 5, prior='uniform')}},
#
#    'lightgbm': {'model': LGBMClassifier(verbose=-1),
#                 'params': {
#                     'learning_rate': Real(1e-3, 1, prior='log-uniform'),
#                     'n_estimators': Integer(10, 500),
#                     'num_leaves': Integer(2, 100),
#                     'max_depth': Integer(3, 10),
#                     'min_child_samples': Integer(1, 50),
#                     'min_child_weight': Real(1e-5, 1e-3, prior='log-uniform'),
#                     'subsample': Real(0.1, 1.0, prior='uniform'),
#                     'colsample_bytree': Real(0.1, 1.0, prior='uniform'),
#                     'reg_alpha': Real(0, 1, prior='uniform'),
#                     'reg_lambda': Real(0, 1, prior='uniform')}},
#
#    'catboost': {'model': CatBoostClassifier(verbose=0),
#                 'params': {
#                     'learning_rate': Real(1e-3, 1, prior='log-uniform'),
#                     'iterations': Integer(10, 500),
#                     'depth': Integer(3, 10),
#                     'l2_leaf_reg': Real(1, 10, prior='uniform'),
#                     'border_count': Integer(1, 255),
#                     'bagging_temperature': Real(0, 1, prior='uniform'),
#                     'random_strength': Real(1e-9, 10, prior='log-uniform')}}
}

In [ ]:
import time
import warnings
from sklearn.exceptions import ConvergenceWarning

# Ignorar FutureWarning
warnings.filterwarnings('ignore', category=FutureWarning)

# Ignorar ConvergenceWarning
warnings.filterwarnings('ignore', category=ConvergenceWarning)

#lista para armazenar a referencia das threads
threads = []

# create a thread pool with max worker threads
pool = ThreadPoolExecutor(max_workers=6)

for model_name, mp in model_params.items():

    # adiciona experimento na lista de threads: pool
    exp = pool.submit(experiment, model_name, mp['model'],mp['params'], X_, y) # does not block

    #adiciona na lista para salvar a referencia da thread
    threads.append(exp)
    time.sleep(0.1)

# aguarda pela finalização das threads
for exp in as_completed(threads):
    exp.result()

# fecha o pool
pool.shutdown